In [2]:
import settings
import algoseek_connector
import algoseek_connector.functions as fn

In [23]:
session = algoseek_connector.Session(
    settings.host, settings.user, settings.password
)

In [24]:
session.ping()

[(1,)]

In [5]:
resource = algoseek_connector.DataResource(session)

In [7]:
for dataset in resource.datagroups.USEquityMarketData.datasets.all():
    print(dataset.name)

BasicAdjustedOHLCDaily
BasicOHLCDaily
PrimaryAdjustedOHLCDaily
PrimaryOHLCDaily
StandardAdjustedOHLCDaily
StandardOHLCDaily
TradeAndQuote
TradeAndQuoteMinuteBar
TradeAndQuoteMinuteBarExcludingTRF
TradeOnly
TradeOnlyAdjusted
TradeOnlyAdjustedMinuteBar
TradeOnlyAdjustedMinuteBarBBG
TradeOnlyAdjustedMinuteBarExcludingTRF
TradeOnlyMinuteBar
TradeOnlyMinuteBarBBG
TradeOnlyMinuteBarExcludingTRF


In [6]:
ds = resource.datagroups.USEquityMarketData.datasets.TradeAndQuote

In [8]:
ds.head().fetch()

,TradeDate,EventDateTime,EventType,Ticker,SecId,Price,Quantity,Exchange,ConditionCode
0,2020-01-02,2020-01-02 06:58:00.160455-05:00,QUOTE BID,A,32952,0,0,BATS Y,1
1,2020-01-02,2020-01-02 06:58:00.160455-05:00,QUOTE ASK,A,32952,0,0,BATS Y,1
2,2020-01-02,2020-01-02 06:58:00.160455-05:00,QUOTE BID NB,A,32952,0,0,UNKNOWN,1
3,2020-01-02,2020-01-02 06:58:00.160455-05:00,QUOTE ASK NB,A,32952,0,0,UNKNOWN,1
4,2020-01-02,2020-01-02 07:00:00.034765-05:00,QUOTE BID NB,A,32952,51.91,100,EDGX,1
5,2020-01-02,2020-01-02 07:00:00.034765-05:00,QUOTE ASK NB,A,32952,86,100,EDGX,1
6,2020-01-02,2020-01-02 07:00:00.035522-05:00,QUOTE BID,A,32952,51.91,100,EDGX,1
7,2020-01-02,2020-01-02 07:00:00.035522-05:00,QUOTE ASK,A,32952,86,100,EDGX,1
8,2020-01-02,2020-01-02 07:00:00.065613-05:00,QUOTE BID NB,A,32952,62,100,EDGX,1
9,2020-01-02,2020-01-02 07:00:00.065613-05:00,QUOTE ASK NB,A,32952,86,100,EDGX,1


In [28]:
ds = resource.datagroups.USEquityMarketData.datasets.TradeOnly

In [29]:
ds.head().fetch()

,TradeDate,EventDateTime,EventType,Ticker,SecId,Price,Quantity,Exchange,ConditionCode
0,2020-01-02,2020-01-02 08:31:06.760379-05:00,TRADE,A,32952,85.4,200,NASDAQ,536879136
1,2020-01-02,2020-01-02 09:29:57.929553-05:00,TRADE,A,32952,85.8,100,ARCA,8192
2,2020-01-02,2020-01-02 09:29:57.930146-05:00,TRADE,A,32952,85.8,100,NASDAQ,8192
3,2020-01-02,2020-01-02 09:30:00.436516-05:00,TRADE NB,A,32952,85.9,24994,NYSE,536870976
4,2020-01-02,2020-01-02 09:30:00.436537-05:00,TRADE,A,32952,85.9,24994,NYSE,67108864
5,2020-01-02,2020-01-02 09:30:00.438162-05:00,TRADE NB,A,32952,85.9,100,NYSE,1
6,2020-01-02,2020-01-02 09:30:00.493193-05:00,TRADE,A,32952,85.55,2,NYSE,2684354592
7,2020-01-02,2020-01-02 09:30:00.493212-05:00,TRADE NB,A,32952,85.52,157,NYSE,536870944
8,2020-01-02,2020-01-02 09:30:00.493242-05:00,TRADE NB,A,32952,85.56,100,AMEX,536870944
9,2020-01-02,2020-01-02 09:30:00.493260-05:00,TRADE,A,32952,85.51,41,NYSE,2684354592


In [30]:
ds.select(ds.Ticker, ds.Price, ds.Quantity).filter(
    (ds.TradeDate == '2020-01-02') & 
    (ds.Exchange == 'FINRA')
).head(1000).fetch()

,Ticker,Price,Quantity
0,KMI,21.02,100
1,KMI,21.0249,525
2,KMI,21.02,96
3,KMI,21.02,97
4,KMI,21.02,200
...,...,...,...
995,KMI,21.06,700
996,KMI,21.06,300
997,KMI,21.06,200
998,KMI,21.06,100


In [43]:
faang = ('FB', 'AAPL', 'AMZN', 'NFLX', 'GOOG')
ds.select([
    ds.Ticker,
    ds.Exchange,
    fn.max(ds.Price).alias('High')
]).filter(
    (ds.Ticker.isin(faang)) &
    (ds.TradeDate == '2020-01-28')
).groupby([
    ds.Ticker,
    ds.Exchange
]).sort(ds.Ticker).fetch()

,Ticker,Exchange,High
0,AAPL,AMEX,327.14
1,AAPL,BATS Y,327.42
2,AAPL,NASDAQ,327.69
3,AAPL,CSE,326.69
4,AAPL,NSE,327.56
...,...,...,...
65,NFLX,NSE,352.1
66,NFLX,AMEX,351.96
67,NFLX,CSE,352.27
68,NFLX,BATS Y,352.29


In [31]:
taq_1min = algoseek_connector.Dataset(
    'USEquityMarketData', 'TradeAndQuoteMinuteBar', session=session
)

In [42]:
taq_1min.select(
    'TradeDate',
    'BarDateTime',
    'Ticker',
    'LastTradePrice'
).filter(
    taq_1min.Ticker.isin(['AMZN', 'GOOG', 'TSLA']) &
    (taq_1min.TradeDate > '2022-01-20') &
    fn.toTime(taq_1min.BarDateTime).between('1970-01-02 09:30:00', '1970-01-02 12:00:00')
).sort('BarDateTime', 'Ticker').head(100).fetch()

,TradeDate,BarDateTime,Ticker,LastTradePrice
0,2022-01-21,2022-01-21 09:30:00-05:00,AMZN,2997.4189
1,2022-01-21,2022-01-21 09:30:00-05:00,GOOG,2663.7087
2,2022-01-21,2022-01-21 09:30:00-05:00,TSLA,1001.16
3,2022-01-21,2022-01-21 09:31:00-05:00,AMZN,3009.1601
4,2022-01-21,2022-01-21 09:31:00-05:00,GOOG,2675.99
...,...,...,...,...
95,2022-01-21,2022-01-21 10:01:00-05:00,TSLA,962.24
96,2022-01-21,2022-01-21 10:02:00-05:00,AMZN,2929.595
97,2022-01-21,2022-01-21 10:02:00-05:00,GOOG,2660.48
98,2022-01-21,2022-01-21 10:02:00-05:00,TSLA,961.59


In [25]:
ohlc = resource.datagroups.USEquityMarketData.datasets.PrimaryAdjustedOHLCDaily

In [26]:
df = ohlc.select([
    ohlc.TradeDate, ohlc.ClosePrice, ohlc.ListedMarketHoursVolume.alias('Volume')
]).filter(
    (ohlc.Ticker == 'IBM') &
    (ohlc.TradeDate >= '2022-01-01') & 
    (ohlc.TradeDate < '2022-04-01')
).fetch()

In [27]:
df

""
